# Práctica 3: Aprendizaje Automático
### 21722 - Inteligencia Artificial
#### Por: Andrea Valentina Reyes Gómez


Se quiere predecir el valor monetario de un jugador de fútbol utilizando la información del mismo.

## Importar librerías

In [1]:
import os

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np

## Análisis de los datos

Como primer paso visualizamos los datos para saber qué debemos cambiar o mantener para que los datos permitan hacer predicciones lo más acertadas posible.

In [2]:
df = pd.read_csv(os.path.join("in", "fifa.csv"))
pd.set_option('display.max_columns', None)
df

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

Al mirar los datos de las columnas nos damos cuenta de que muchos de ellos son inválidos o irrelevantes en la decisión final del valor monetario de un jugador. Por lo tanto, procedemos a identificar y eliminar dichas columnas.

Las columnas **Unnamed**, **ID**, **Photo**, **Flag**, **Club Logo**, **Special**, **Preferred Foot**, **Real Face**, **Jersey Number**, **Joined** y **Contract Valid Until** son las que no proporcionan información relevante, puesto que no son datos medibles ni le suman o restan valor a un jugador. 

In [3]:
df.pop('Unnamed: 0')
df.pop('ID')
df.pop('Photo')
df.pop('Flag')
df.pop('Club Logo')
df.pop('Special')
df.pop("Preferred Foot")
df.pop('Real Face')
df.pop('Jersey Number')
df.pop('Joined')
df.pop('Contract Valid Until')

0        2021
1        2022
2        2022
3        2020
4        2023
         ... 
18202    2019
18203    2020
18204    2021
18205    2019
18206    2019
Name: Contract Valid Until, Length: 18207, dtype: object

Ahora observamos la tabla luego de eliminar algunos datos irrelevantes. 

In [4]:
df

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Loaned From,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,L. Messi,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,5.0,4.0,4.0,Medium/ Medium,Messi,RF,NaN,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,Cristiano Ronaldo,33,Portugal,94,94,Juventus,€77M,€405K,5.0,4.0,5.0,High/ Low,C. Ronaldo,ST,NaN,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,5.0,5.0,5.0,High/ Medium,Neymar,LW,NaN,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,De Gea,27,Spain,91,93,Manchester United,€72M,€260K,4.0,3.0,1.0,Medium/ Medium,Lean,GK,NaN,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,K. De Bruyne,27,Belgium,91,92,Manchester City,€102M,€355K,4.0,5.0,4.0,High/ High,Normal,RCM,NaN,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,J. Lundstram,19,England,47,65,Crewe Alexandra,€60K,€1K,1.0,2.0,2.0,Medium/ Medium,Lean,CM,NaN,5'9,134lbs,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,€143K
18203,N. Christoffersson,19,Sweden,47,63,Trelleborgs FF,€60K,€1K,1.0,2.0,2.0,Medium/ Medium,Normal,ST,NaN,6'3,170lbs,45+2,45+2,45+2,39+2,42+2,42+2,42+2,39+2,40+2,40+2,40+2,38+2,35+2,35+2,35+2,38+2,30+2,31+2,31+2,31+2,30+2,29+2,32+2,32+2,32+2,29+2,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.0,25.0,40.0,41.0,39.0,38.0,40.0,52.0,41.0,47.0,43.0,67.0,42.0,47.0,16.0,46.0,33.0,43.0,42.0,22.0,15.0,19.0,10.0,9.0,9.0,5.0,12.0,€113K
18204,B. Worman,16,England,47,67,Cambridge United,€60K,€1K,1.0,3.0,2.0,Medium/ Medium,Normal,ST,NaN,5'8,148lbs,45+2,45+2,45+2,45+2,46+2,46+2,46+2,45+2,44+2,44+2,44+2,44+2,38+2,38

## Tratamiento de valores no numéricos

Podemos observar que hay ciertos datos que corresponden a valores monetarios, pero que en sus instancias incluyen el símbolo de la moneda en la que está expresado dicho valor. Como todos estos valores están expresados en euros, podemos remover el símbolo y así tener solo datos numéricos que nos permitan hacer mejores estimaciones.

Además, sabemos que K indica que el valor está en el rango de los miles y que M indica que está en el rango de los millones, por lo que también podemos eliminar estas letras y colocar su valor correspondiente.

Esto se realizará utilizando el código de transformación proporcionado en el enunciado de la práctica.

In [5]:
def value_to_float(x):
    """
    From K and M to float.
    Delete '€'
    
    """
    x = x.replace('€', '')
    ret_val = 0.0
    
    if type(x) == float or type(x) == int:
        ret_val = x
    if 'K' in x:
        if len(x) > 1:
            ret_val = float(x.replace('K', ''))
        ret_val = ret_val *1000
    if 'M' in x:
        if len(x) > 1:
            ret_val = float(x.replace('M', ''))
        ret_val = ret_val * 1000000.0
    return ret_val

Aplicamos esta función a cada valor de la columna **Value**.

In [6]:
df["Value"] = df["Value"].apply(value_to_float)
df.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Loaned From,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,L. Messi,31,Argentina,94,94,FC Barcelona,110500000.0,€565K,5.0,4.0,4.0,Medium/ Medium,Messi,RF,NaN,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,Cristiano Ronaldo,33,Portugal,94,94,Juventus,77000000.0,€405K,5.0,4.0,5.0,High/ Low,C. Ronaldo,ST,NaN,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,118500000.0,€290K,5.0,5.0,5.0,High/ Medium,Neymar,LW,NaN,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,De Gea,27,Spain,91,93,Manchester United,72000000.0,€260K,4.0,3.0,1.0,Medium/ Medium,Lean,GK,NaN,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,K. De Bruyne,27,Belgium,91,92,Manchester City,102000000.0,€355K,4.0,5.0,4.0,High/ High,Normal,RCM,NaN,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


Aplicamos la misma función al resto de columnas que contienen valores monetarios: **Wage** y **Release Clause**.

In [7]:
df["Wage"] = df["Wage"].apply(value_to_float)
df.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Loaned From,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,L. Messi,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,5.0,4.0,4.0,Medium/ Medium,Messi,RF,NaN,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,Cristiano Ronaldo,33,Portugal,94,94,Juventus,77000000.0,405000.0,5.0,4.0,5.0,High/ Low,C. Ronaldo,ST,NaN,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,5.0,5.0,5.0,High/ Medium,Neymar,LW,NaN,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,De Gea,27,Spain,91,93,Manchester United,72000000.0,260000.0,4.0,3.0,1.0,Medium/ Medium,Lean,GK,NaN,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,K. De Bruyne,27,Belgium,91,92,Manchester City,102000000.0,355000.0,4.0,5.0,4.0,High/ High,Normal,RCM,NaN,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


Cambiamos el tipo de datos de la columna **Release Clause** a *str* y aplicamos la función de transformación definida anteriormente.

In [8]:
df["Release Clause"] = df["Release Clause"].astype(str)

In [9]:
df["Release Clause"] = df["Release Clause"].apply(value_to_float)
df.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Loaned From,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,L. Messi,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,5.0,4.0,4.0,Medium/ Medium,Messi,RF,NaN,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0
1,Cristiano Ronaldo,33,Portugal,94,94,Juventus,77000000.0,405000.0,5.0,4.0,5.0,High/ Low,C. Ronaldo,ST,NaN,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0
2,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,5.0,5.0,5.0,High/ Medium,Neymar,LW,NaN,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0
3,De Gea,27,Spain,91,93,Manchester United,72000000.0,260000.0,4.0,3.0,1.0,Medium/ Medium,Lean,GK,NaN,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0
4,K. De Bruyne,27,Belgium,91,92,Manchester City,102000000.0,355000.0,4.0,5.0,4.0,High/ High,Normal,RCM,NaN,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0


Vemos que en la columna **Weight** los valores tienen como sufijo *'lbs'* para indicar que se mide en libras. Como la unidad es la misma para todos los pesos, eliminamos el sufijo para quedarnos solo con el valor numérico.

In [10]:
def replace_lbs(x):
    """
    Delete 'lbs' from weight
    
    """
    x = x.replace('lbs', '')
    
    return x

In [11]:
df["Weight"] = df["Weight"].astype(str)

In [12]:
df["Weight"] = df["Weight"].apply(replace_lbs)
df.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Loaned From,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,L. Messi,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,5.0,4.0,4.0,Medium/ Medium,Messi,RF,NaN,5'7,159,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0
1,Cristiano Ronaldo,33,Portugal,94,94,Juventus,77000000.0,405000.0,5.0,4.0,5.0,High/ Low,C. Ronaldo,ST,NaN,6'2,183,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0
2,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,5.0,5.0,5.0,High/ Medium,Neymar,LW,NaN,5'9,150,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0
3,De Gea,27,Spain,91,93,Manchester United,72000000.0,260000.0,4.0,3.0,1.0,Medium/ Medium,Lean,GK,NaN,6'4,168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0
4,K. De Bruyne,27,Belgium,91,92,Manchester City,102000000.0,355000.0,4.0,5.0,4.0,High/ High,Normal,RCM,NaN,5'11,154,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0


Por su parte, la columna **Height** tiene valores con un apóstrofe en lugar de un punto. Lo reemplazaremos para convertir este valor en numérico.

In [13]:
def replace_apostrophe(x):
    """
    Delete apostrophe from height
    
    """
    x = x.replace('\'', '.')
    
    return x

In [14]:
df["Height"] = df["Height"].astype(str)

In [15]:
df["Height"] = df["Height"].apply(replace_apostrophe)
df.head()

,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Loaned From,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,L. Messi,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,5.0,4.0,4.0,Medium/ Medium,Messi,RF,NaN,5.7,159,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0
1,Cristiano Ronaldo,33,Portugal,94,94,Juventus,77000000.0,405000.0,5.0,4.0,5.0,High/ Low,C. Ronaldo,ST,NaN,6.2,183,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0
2,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,5.0,5.0,5.0,High/ Medium,Neymar,LW,NaN,5.9,150,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0
3,De Gea,27,Spain,91,93,Manchester United,72000000.0,260000.0,4.0,3.0,1.0,Medium/ Medium,Lean,GK,NaN,6.4,168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0
4,K. De Bruyne,27,Belgium,91,92,Manchester City,102000000.0,355000.0,4.0,5.0,4.0,High/ High,Normal,RCM,NaN,5.11,154,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0


Otra característica que influye en el precio de un futbolista es su **nacionalidad**, ya que para ciertas competiciones se limita a los clubes en cuando a la cantidad de jugadores extranjeros, etc. Sin embargo, los valores de esta característica no están presentados de forma numérica en este set de datos, por lo cual transformaremos esta columna en varias, una para cada nacionalidad y colocaremos un 1 si el jugador es de esa nacionalidad o un 0 en caso contrario.

In [16]:
nationality = df.pop("Nationality")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(nationality, prefix='Nationality').reset_index(drop=True)], axis=1, sort=False)
df.head()

,Name,Age,Overall,Potential,Club,Value,Wage,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Loaned From,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Nationality_Afghanistan,Nationality_Albania,Nationality_Algeria,Nationality_Andorra,Nationality_Angola,Nationality_Antigua & Barbuda,Nationality_Argentina,Nationality_Armenia,Nationality_Australia,Nationality_Austria,Nationality_Azerbaijan,Nationality_Barbados,Nationality_Belarus,Nationality_Belgium,Nationality_Belize,Nationality_Benin,Nationality_Bermuda,Nationality_Bolivia,Nationality_Bosnia Herzegovina,Nationality_Botswana,Nationality_Brazil,Nationality_Bulgaria,Nationality_Burkina Faso,Nationality_Burundi,Nationality_Cameroon,Nationality_Canada,Nationality_Cape Verde,Nationality_Central African Rep.,Nationality_Chad,Nationality_Chile,Nationality_China PR,Nationality_Colombia,Nationality_Comoros,Nationality_Congo,Nationality_Costa Rica,Nationality_Croatia,Nationality_Cuba,Nationality_Curacao,Nationality_Cyprus,Nationality_Czech Republic,Nationality_DR Congo,Nationality_Denmark,Nationality_Dominican Republic,Nationality_Ecuador,Nationality_Egypt,Nationality_El Salvador,Nationality_England,Nationality_Equatorial Guinea,Nationality_Eritrea,Nationality_Estonia,Nationality_Ethiopia,Nationality_FYR Macedonia,Nationality_Faroe Islands,Nationality_Fiji,Nationality_Finland,Nationality_France,Nationality_Gabon,Nationality_Gambia,Nationality_Georgia,Nationality_Germany,Nationality_Ghana,Nationality_Greece,Nationality_Grenada,Nationality_Guam,Nationality_Guatemala,Nationality_Guinea,Nationality_Guinea Bissau,Nationality_Guyana,Nationality_Haiti,Nationality_Honduras,Nationality_Hong Kong,Nationality_Hungary,Nationality_Iceland,Nationality_India,Nationality_Indonesia,Nationality_Iran,Nationality_Iraq,Nationality_Israel,Nationality_Italy,Nationality_Ivory Coast,Nationality_Jamaica,Nationality_Japan,Nationality_Jordan,Nationality_Kazakhstan,Nationality_Kenya,Nationality_Korea DPR,Nationality_Korea Republic,Nationality_Kosovo,Nationality_Kuwait,Nationality_Latvia,Nationality_Lebanon,Nationality_Liberia,Nationality_Libya,Nationality_Liechtenstein,Nationality_Lithuania,Nationality_Luxembourg,Nationality_Madagascar,Nationality_Mali,Nationality_Malta,Nationality_Mauritania,Nationality_Mauritius,Nationality_Mexico,Nationality_Moldova,Nationality_Montenegro,Nationality_Montserrat,Nationality_Morocco,Nationality_Mozambique,Nationality_Namibia,Nationality_Netherlands,Nationality_New Caledonia,Nationality_New Zealand,Nationality_Nicaragua,Nationality_Niger,Nationality_Nigeria,Nationality_Northern Ireland,Nationality_Norway,Nationality_Oman,Nationality_Palestine,Nationality_Panama,Nationality_Paraguay,Nationality_Peru,Nationality_Philippines,Nationality_Poland,Nationality_Portugal,Nationality_Puerto Rico,Nationality_Qatar,Nationality_Republic of Ireland,Nationality_Romania,Nationality_Russia,Nationality_Rwanda,Nationality_Saudi Arabia,Nationality_Scotland,Nationality_Senegal,Nationality_Serbia,Nationality_Sierra Leone,Nationality_Slovakia,Nationality_Slovenia,Nationality_South Africa,Nationality_South Sudan,Nationality_Spain,Nationality_St Kitts Nevis,Nationality_St Lucia,Nationality_Sudan,Nationality_Suriname,Nationality_Sweden,Nationality_Switzerland,Nationality_Syria,Nationality_São Tomé & Príncipe,Nationality_Tanzania,Nationality_Thailand,Nationality_Togo,Nationality_Trinidad & Tobago,Nationality_Tunisia,Nationality_Turkey,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationali

## Tratamiento de valores NaN

Ahora nos dedicamos a tratar aquellos datos que tienen valores inválidos. Para ello, primero buscamos a qué columnas pertenecen esos valores

In [17]:
df.columns[df.isna().any()].tolist()

['Club',
 'International Reputation',
 'Weak Foot',
 'Skill Moves',
 'Work Rate',
 'Body Type',
 'Position',
 'Loaned From',
 'LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM',
 'CAM',
 'RAM',
 'LM',
 'LCM',
 'CM',
 'RCM',
 'RM',
 'LWB',
 'LDM',
 'CDM',
 'RDM',
 'RWB',
 'LB',
 'LCB',
 'CB',
 'RCB',
 'RB',
 'Crossing',
 'Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Reactions',
 'Balance',
 'ShotPower',
 'Jumping',
 'Stamina',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Marking',
 'StandingTackle',
 'SlidingTackle',
 'GKDiving',
 'GKHandling',
 'GKKicking',
 'GKPositioning',
 'GKReflexes']

In [18]:
df[df.isna().any(axis=1)]

,Name,Age,Overall,Potential,Club,Value,Wage,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Loaned From,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Nationality_Afghanistan,Nationality_Albania,Nationality_Algeria,Nationality_Andorra,Nationality_Angola,Nationality_Antigua & Barbuda,Nationality_Argentina,Nationality_Armenia,Nationality_Australia,Nationality_Austria,Nationality_Azerbaijan,Nationality_Barbados,Nationality_Belarus,Nationality_Belgium,Nationality_Belize,Nationality_Benin,Nationality_Bermuda,Nationality_Bolivia,Nationality_Bosnia Herzegovina,Nationality_Botswana,Nationality_Brazil,Nationality_Bulgaria,Nationality_Burkina Faso,Nationality_Burundi,Nationality_Cameroon,Nationality_Canada,Nationality_Cape Verde,Nationality_Central African Rep.,Nationality_Chad,Nationality_Chile,Nationality_China PR,Nationality_Colombia,Nationality_Comoros,Nationality_Congo,Nationality_Costa Rica,Nationality_Croatia,Nationality_Cuba,Nationality_Curacao,Nationality_Cyprus,Nationality_Czech Republic,Nationality_DR Congo,Nationality_Denmark,Nationality_Dominican Republic,Nationality_Ecuador,Nationality_Egypt,Nationality_El Salvador,Nationality_England,Nationality_Equatorial Guinea,Nationality_Eritrea,Nationality_Estonia,Nationality_Ethiopia,Nationality_FYR Macedonia,Nationality_Faroe Islands,Nationality_Fiji,Nationality_Finland,Nationality_France,Nationality_Gabon,Nationality_Gambia,Nationality_Georgia,Nationality_Germany,Nationality_Ghana,Nationality_Greece,Nationality_Grenada,Nationality_Guam,Nationality_Guatemala,Nationality_Guinea,Nationality_Guinea Bissau,Nationality_Guyana,Nationality_Haiti,Nationality_Honduras,Nationality_Hong Kong,Nationality_Hungary,Nationality_Iceland,Nationality_India,Nationality_Indonesia,Nationality_Iran,Nationality_Iraq,Nationality_Israel,Nationality_Italy,Nationality_Ivory Coast,Nationality_Jamaica,Nationality_Japan,Nationality_Jordan,Nationality_Kazakhstan,Nationality_Kenya,Nationality_Korea DPR,Nationality_Korea Republic,Nationality_Kosovo,Nationality_Kuwait,Nationality_Latvia,Nationality_Lebanon,Nationality_Liberia,Nationality_Libya,Nationality_Liechtenstein,Nationality_Lithuania,Nationality_Luxembourg,Nationality_Madagascar,Nationality_Mali,Nationality_Malta,Nationality_Mauritania,Nationality_Mauritius,Nationality_Mexico,Nationality_Moldova,Nationality_Montenegro,Nationality_Montserrat,Nationality_Morocco,Nationality_Mozambique,Nationality_Namibia,Nationality_Netherlands,Nationality_New Caledonia,Nationality_New Zealand,Nationality_Nicaragua,Nationality_Niger,Nationality_Nigeria,Nationality_Northern Ireland,Nationality_Norway,Nationality_Oman,Nationality_Palestine,Nationality_Panama,Nationality_Paraguay,Nationality_Peru,Nationality_Philippines,Nationality_Poland,Nationality_Portugal,Nationality_Puerto Rico,Nationality_Qatar,Nationality_Republic of Ireland,Nationality_Romania,Nationality_Russia,Nationality_Rwanda,Nationality_Saudi Arabia,Nationality_Scotland,Nationality_Senegal,Nationality_Serbia,Nationality_Sierra Leone,Nationality_Slovakia,Nationality_Slovenia,Nationality_South Africa,Nationality_South Sudan,Nationality_Spain,Nationality_St Kitts Nevis,Nationality_St Lucia,Nationality_Sudan,Nationality_Suriname,Nationality_Sweden,Nationality_Switzerland,Nationality_Syria,Nationality_São Tomé & Príncipe,Nationality_Tanzania,Nationality_Thailand,Nationality_Togo,Nationality_Trinidad & Tobago,Nationality_Tunisia,Nationality_Turkey,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationali

De aquí podemos observar que la mayoría de las columnas tienen al menos un valor NaN (Not a Number). 

Para tratar estos datos podemos empezar por eliminar la columna **Loaned From**, ya que no aporta información relevante.

In [19]:
df.pop("Loaned From")
df

,Name,Age,Overall,Potential,Club,Value,Wage,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Nationality_Afghanistan,Nationality_Albania,Nationality_Algeria,Nationality_Andorra,Nationality_Angola,Nationality_Antigua & Barbuda,Nationality_Argentina,Nationality_Armenia,Nationality_Australia,Nationality_Austria,Nationality_Azerbaijan,Nationality_Barbados,Nationality_Belarus,Nationality_Belgium,Nationality_Belize,Nationality_Benin,Nationality_Bermuda,Nationality_Bolivia,Nationality_Bosnia Herzegovina,Nationality_Botswana,Nationality_Brazil,Nationality_Bulgaria,Nationality_Burkina Faso,Nationality_Burundi,Nationality_Cameroon,Nationality_Canada,Nationality_Cape Verde,Nationality_Central African Rep.,Nationality_Chad,Nationality_Chile,Nationality_China PR,Nationality_Colombia,Nationality_Comoros,Nationality_Congo,Nationality_Costa Rica,Nationality_Croatia,Nationality_Cuba,Nationality_Curacao,Nationality_Cyprus,Nationality_Czech Republic,Nationality_DR Congo,Nationality_Denmark,Nationality_Dominican Republic,Nationality_Ecuador,Nationality_Egypt,Nationality_El Salvador,Nationality_England,Nationality_Equatorial Guinea,Nationality_Eritrea,Nationality_Estonia,Nationality_Ethiopia,Nationality_FYR Macedonia,Nationality_Faroe Islands,Nationality_Fiji,Nationality_Finland,Nationality_France,Nationality_Gabon,Nationality_Gambia,Nationality_Georgia,Nationality_Germany,Nationality_Ghana,Nationality_Greece,Nationality_Grenada,Nationality_Guam,Nationality_Guatemala,Nationality_Guinea,Nationality_Guinea Bissau,Nationality_Guyana,Nationality_Haiti,Nationality_Honduras,Nationality_Hong Kong,Nationality_Hungary,Nationality_Iceland,Nationality_India,Nationality_Indonesia,Nationality_Iran,Nationality_Iraq,Nationality_Israel,Nationality_Italy,Nationality_Ivory Coast,Nationality_Jamaica,Nationality_Japan,Nationality_Jordan,Nationality_Kazakhstan,Nationality_Kenya,Nationality_Korea DPR,Nationality_Korea Republic,Nationality_Kosovo,Nationality_Kuwait,Nationality_Latvia,Nationality_Lebanon,Nationality_Liberia,Nationality_Libya,Nationality_Liechtenstein,Nationality_Lithuania,Nationality_Luxembourg,Nationality_Madagascar,Nationality_Mali,Nationality_Malta,Nationality_Mauritania,Nationality_Mauritius,Nationality_Mexico,Nationality_Moldova,Nationality_Montenegro,Nationality_Montserrat,Nationality_Morocco,Nationality_Mozambique,Nationality_Namibia,Nationality_Netherlands,Nationality_New Caledonia,Nationality_New Zealand,Nationality_Nicaragua,Nationality_Niger,Nationality_Nigeria,Nationality_Northern Ireland,Nationality_Norway,Nationality_Oman,Nationality_Palestine,Nationality_Panama,Nationality_Paraguay,Nationality_Peru,Nationality_Philippines,Nationality_Poland,Nationality_Portugal,Nationality_Puerto Rico,Nationality_Qatar,Nationality_Republic of Ireland,Nationality_Romania,Nationality_Russia,Nationality_Rwanda,Nationality_Saudi Arabia,Nationality_Scotland,Nationality_Senegal,Nationality_Serbia,Nationality_Sierra Leone,Nationality_Slovakia,Nationality_Slovenia,Nationality_South Africa,Nationality_South Sudan,Nationality_Spain,Nationality_St Kitts Nevis,Nationality_St Lucia,Nationality_Sudan,Nationality_Suriname,Nationality_Sweden,Nationality_Switzerland,Nationality_Syria,Nationality_São Tomé & Príncipe,Nationality_Tanzania,Nationality_Thailand,Nationality_Togo,Nationality_Trinidad & Tobago,Nationality_Tunisia,Nationality_Turkey,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationality_Uzbekista

Ahora, analizamos para cada columna aquellas casillas que tengan valores NaN y veremos cómo tratar cada una de ellas. Empezamos con el atributo **International Reputation**

In [20]:
df.loc[df["International Reputation"] != df["International Reputation"]]

,Name,Age,Overall,Potential,Club,Value,Wage,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Nationality_Afghanistan,Nationality_Albania,Nationality_Algeria,Nationality_Andorra,Nationality_Angola,Nationality_Antigua & Barbuda,Nationality_Argentina,Nationality_Armenia,Nationality_Australia,Nationality_Austria,Nationality_Azerbaijan,Nationality_Barbados,Nationality_Belarus,Nationality_Belgium,Nationality_Belize,Nationality_Benin,Nationality_Bermuda,Nationality_Bolivia,Nationality_Bosnia Herzegovina,Nationality_Botswana,Nationality_Brazil,Nationality_Bulgaria,Nationality_Burkina Faso,Nationality_Burundi,Nationality_Cameroon,Nationality_Canada,Nationality_Cape Verde,Nationality_Central African Rep.,Nationality_Chad,Nationality_Chile,Nationality_China PR,Nationality_Colombia,Nationality_Comoros,Nationality_Congo,Nationality_Costa Rica,Nationality_Croatia,Nationality_Cuba,Nationality_Curacao,Nationality_Cyprus,Nationality_Czech Republic,Nationality_DR Congo,Nationality_Denmark,Nationality_Dominican Republic,Nationality_Ecuador,Nationality_Egypt,Nationality_El Salvador,Nationality_England,Nationality_Equatorial Guinea,Nationality_Eritrea,Nationality_Estonia,Nationality_Ethiopia,Nationality_FYR Macedonia,Nationality_Faroe Islands,Nationality_Fiji,Nationality_Finland,Nationality_France,Nationality_Gabon,Nationality_Gambia,Nationality_Georgia,Nationality_Germany,Nationality_Ghana,Nationality_Greece,Nationality_Grenada,Nationality_Guam,Nationality_Guatemala,Nationality_Guinea,Nationality_Guinea Bissau,Nationality_Guyana,Nationality_Haiti,Nationality_Honduras,Nationality_Hong Kong,Nationality_Hungary,Nationality_Iceland,Nationality_India,Nationality_Indonesia,Nationality_Iran,Nationality_Iraq,Nationality_Israel,Nationality_Italy,Nationality_Ivory Coast,Nationality_Jamaica,Nationality_Japan,Nationality_Jordan,Nationality_Kazakhstan,Nationality_Kenya,Nationality_Korea DPR,Nationality_Korea Republic,Nationality_Kosovo,Nationality_Kuwait,Nationality_Latvia,Nationality_Lebanon,Nationality_Liberia,Nationality_Libya,Nationality_Liechtenstein,Nationality_Lithuania,Nationality_Luxembourg,Nationality_Madagascar,Nationality_Mali,Nationality_Malta,Nationality_Mauritania,Nationality_Mauritius,Nationality_Mexico,Nationality_Moldova,Nationality_Montenegro,Nationality_Montserrat,Nationality_Morocco,Nationality_Mozambique,Nationality_Namibia,Nationality_Netherlands,Nationality_New Caledonia,Nationality_New Zealand,Nationality_Nicaragua,Nationality_Niger,Nationality_Nigeria,Nationality_Northern Ireland,Nationality_Norway,Nationality_Oman,Nationality_Palestine,Nationality_Panama,Nationality_Paraguay,Nationality_Peru,Nationality_Philippines,Nationality_Poland,Nationality_Portugal,Nationality_Puerto Rico,Nationality_Qatar,Nationality_Republic of Ireland,Nationality_Romania,Nationality_Russia,Nationality_Rwanda,Nationality_Saudi Arabia,Nationality_Scotland,Nationality_Senegal,Nationality_Serbia,Nationality_Sierra Leone,Nationality_Slovakia,Nationality_Slovenia,Nationality_South Africa,Nationality_South Sudan,Nationality_Spain,Nationality_St Kitts Nevis,Nationality_St Lucia,Nationality_Sudan,Nationality_Suriname,Nationality_Sweden,Nationality_Switzerland,Nationality_Syria,Nationality_São Tomé & Príncipe,Nationality_Tanzania,Nationality_Thailand,Nationality_Togo,Nationality_Trinidad & Tobago,Nationality_Tunisia,Nationality_Turkey,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationality_Uzbekista

Se observa que al menos 48 filas tienen valores NaN para más de la mitad de las características de nuestro set de datos. Como 48 es equivalente a un porcentaje bastante pequeño de 18207 datos que hay, podemos eliminar estas filas problemáticas.

In [21]:
df = df.dropna(subset = ['International Reputation'])
df

,Name,Age,Overall,Potential,Club,Value,Wage,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Nationality_Afghanistan,Nationality_Albania,Nationality_Algeria,Nationality_Andorra,Nationality_Angola,Nationality_Antigua & Barbuda,Nationality_Argentina,Nationality_Armenia,Nationality_Australia,Nationality_Austria,Nationality_Azerbaijan,Nationality_Barbados,Nationality_Belarus,Nationality_Belgium,Nationality_Belize,Nationality_Benin,Nationality_Bermuda,Nationality_Bolivia,Nationality_Bosnia Herzegovina,Nationality_Botswana,Nationality_Brazil,Nationality_Bulgaria,Nationality_Burkina Faso,Nationality_Burundi,Nationality_Cameroon,Nationality_Canada,Nationality_Cape Verde,Nationality_Central African Rep.,Nationality_Chad,Nationality_Chile,Nationality_China PR,Nationality_Colombia,Nationality_Comoros,Nationality_Congo,Nationality_Costa Rica,Nationality_Croatia,Nationality_Cuba,Nationality_Curacao,Nationality_Cyprus,Nationality_Czech Republic,Nationality_DR Congo,Nationality_Denmark,Nationality_Dominican Republic,Nationality_Ecuador,Nationality_Egypt,Nationality_El Salvador,Nationality_England,Nationality_Equatorial Guinea,Nationality_Eritrea,Nationality_Estonia,Nationality_Ethiopia,Nationality_FYR Macedonia,Nationality_Faroe Islands,Nationality_Fiji,Nationality_Finland,Nationality_France,Nationality_Gabon,Nationality_Gambia,Nationality_Georgia,Nationality_Germany,Nationality_Ghana,Nationality_Greece,Nationality_Grenada,Nationality_Guam,Nationality_Guatemala,Nationality_Guinea,Nationality_Guinea Bissau,Nationality_Guyana,Nationality_Haiti,Nationality_Honduras,Nationality_Hong Kong,Nationality_Hungary,Nationality_Iceland,Nationality_India,Nationality_Indonesia,Nationality_Iran,Nationality_Iraq,Nationality_Israel,Nationality_Italy,Nationality_Ivory Coast,Nationality_Jamaica,Nationality_Japan,Nationality_Jordan,Nationality_Kazakhstan,Nationality_Kenya,Nationality_Korea DPR,Nationality_Korea Republic,Nationality_Kosovo,Nationality_Kuwait,Nationality_Latvia,Nationality_Lebanon,Nationality_Liberia,Nationality_Libya,Nationality_Liechtenstein,Nationality_Lithuania,Nationality_Luxembourg,Nationality_Madagascar,Nationality_Mali,Nationality_Malta,Nationality_Mauritania,Nationality_Mauritius,Nationality_Mexico,Nationality_Moldova,Nationality_Montenegro,Nationality_Montserrat,Nationality_Morocco,Nationality_Mozambique,Nationality_Namibia,Nationality_Netherlands,Nationality_New Caledonia,Nationality_New Zealand,Nationality_Nicaragua,Nationality_Niger,Nationality_Nigeria,Nationality_Northern Ireland,Nationality_Norway,Nationality_Oman,Nationality_Palestine,Nationality_Panama,Nationality_Paraguay,Nationality_Peru,Nationality_Philippines,Nationality_Poland,Nationality_Portugal,Nationality_Puerto Rico,Nationality_Qatar,Nationality_Republic of Ireland,Nationality_Romania,Nationality_Russia,Nationality_Rwanda,Nationality_Saudi Arabia,Nationality_Scotland,Nationality_Senegal,Nationality_Serbia,Nationality_Sierra Leone,Nationality_Slovakia,Nationality_Slovenia,Nationality_South Africa,Nationality_South Sudan,Nationality_Spain,Nationality_St Kitts Nevis,Nationality_St Lucia,Nationality_Sudan,Nationality_Suriname,Nationality_Sweden,Nationality_Switzerland,Nationality_Syria,Nationality_São Tomé & Príncipe,Nationality_Tanzania,Nationality_Thailand,Nationality_Togo,Nationality_Trinidad & Tobago,Nationality_Tunisia,Nationality_Turkey,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationality_Uzbekista

Luego, observamos qué columnas siguen afectadas por valores NaN.

In [22]:
df.columns[df.isna().any()].tolist()

['Club',
 'Position',
 'LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM',
 'CAM',
 'RAM',
 'LM',
 'LCM',
 'CM',
 'RCM',
 'RM',
 'LWB',
 'LDM',
 'CDM',
 'RDM',
 'RWB',
 'LB',
 'LCB',
 'CB',
 'RCB',
 'RB']

Como resultado de la limpieza ahora tenemos un menos número de columnas que contienen valores NaN. A partir de la 4ta característica que tiene valores NaN podermos tratarlos calculando la media que de ese dato y usándola para sustituir el inválido.

Otra columna que nos da problemas por tener valores NaN es **Club**, pero como este atributo tiene peso en el valor de un jugador, no lo eliminaremos. En cambio, lo transformaremos de manera que podamos obtener valores numéricos sobre esta información para poder utilizarla en nuestro algoritmo.

La transformación consiste en añadir a la tabla el nombre de cada club y colocar un 1 si el jugador pertenece al club o un 0 en caso contrario.

In [23]:
df.loc[df.Club != df.Club]

,Name,Age,Overall,Potential,Club,Value,Wage,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Nationality_Afghanistan,Nationality_Albania,Nationality_Algeria,Nationality_Andorra,Nationality_Angola,Nationality_Antigua & Barbuda,Nationality_Argentina,Nationality_Armenia,Nationality_Australia,Nationality_Austria,Nationality_Azerbaijan,Nationality_Barbados,Nationality_Belarus,Nationality_Belgium,Nationality_Belize,Nationality_Benin,Nationality_Bermuda,Nationality_Bolivia,Nationality_Bosnia Herzegovina,Nationality_Botswana,Nationality_Brazil,Nationality_Bulgaria,Nationality_Burkina Faso,Nationality_Burundi,Nationality_Cameroon,Nationality_Canada,Nationality_Cape Verde,Nationality_Central African Rep.,Nationality_Chad,Nationality_Chile,Nationality_China PR,Nationality_Colombia,Nationality_Comoros,Nationality_Congo,Nationality_Costa Rica,Nationality_Croatia,Nationality_Cuba,Nationality_Curacao,Nationality_Cyprus,Nationality_Czech Republic,Nationality_DR Congo,Nationality_Denmark,Nationality_Dominican Republic,Nationality_Ecuador,Nationality_Egypt,Nationality_El Salvador,Nationality_England,Nationality_Equatorial Guinea,Nationality_Eritrea,Nationality_Estonia,Nationality_Ethiopia,Nationality_FYR Macedonia,Nationality_Faroe Islands,Nationality_Fiji,Nationality_Finland,Nationality_France,Nationality_Gabon,Nationality_Gambia,Nationality_Georgia,Nationality_Germany,Nationality_Ghana,Nationality_Greece,Nationality_Grenada,Nationality_Guam,Nationality_Guatemala,Nationality_Guinea,Nationality_Guinea Bissau,Nationality_Guyana,Nationality_Haiti,Nationality_Honduras,Nationality_Hong Kong,Nationality_Hungary,Nationality_Iceland,Nationality_India,Nationality_Indonesia,Nationality_Iran,Nationality_Iraq,Nationality_Israel,Nationality_Italy,Nationality_Ivory Coast,Nationality_Jamaica,Nationality_Japan,Nationality_Jordan,Nationality_Kazakhstan,Nationality_Kenya,Nationality_Korea DPR,Nationality_Korea Republic,Nationality_Kosovo,Nationality_Kuwait,Nationality_Latvia,Nationality_Lebanon,Nationality_Liberia,Nationality_Libya,Nationality_Liechtenstein,Nationality_Lithuania,Nationality_Luxembourg,Nationality_Madagascar,Nationality_Mali,Nationality_Malta,Nationality_Mauritania,Nationality_Mauritius,Nationality_Mexico,Nationality_Moldova,Nationality_Montenegro,Nationality_Montserrat,Nationality_Morocco,Nationality_Mozambique,Nationality_Namibia,Nationality_Netherlands,Nationality_New Caledonia,Nationality_New Zealand,Nationality_Nicaragua,Nationality_Niger,Nationality_Nigeria,Nationality_Northern Ireland,Nationality_Norway,Nationality_Oman,Nationality_Palestine,Nationality_Panama,Nationality_Paraguay,Nationality_Peru,Nationality_Philippines,Nationality_Poland,Nationality_Portugal,Nationality_Puerto Rico,Nationality_Qatar,Nationality_Republic of Ireland,Nationality_Romania,Nationality_Russia,Nationality_Rwanda,Nationality_Saudi Arabia,Nationality_Scotland,Nationality_Senegal,Nationality_Serbia,Nationality_Sierra Leone,Nationality_Slovakia,Nationality_Slovenia,Nationality_South Africa,Nationality_South Sudan,Nationality_Spain,Nationality_St Kitts Nevis,Nationality_St Lucia,Nationality_Sudan,Nationality_Suriname,Nationality_Sweden,Nationality_Switzerland,Nationality_Syria,Nationality_São Tomé & Príncipe,Nationality_Tanzania,Nationality_Thailand,Nationality_Togo,Nationality_Trinidad & Tobago,Nationality_Tunisia,Nationality_Turkey,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationality_Uzbekista

In [24]:
df = df.dropna(subset = ['Club'])

In [25]:
clb = df.pop("Club")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(clb, prefix='clb').reset_index(drop=True)], axis=1, sort=False)
df

,Name,Age,Overall,Potential,Value,Wage,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Nationality_Afghanistan,Nationality_Albania,Nationality_Algeria,Nationality_Andorra,Nationality_Angola,Nationality_Antigua & Barbuda,Nationality_Argentina,Nationality_Armenia,Nationality_Australia,Nationality_Austria,Nationality_Azerbaijan,Nationality_Barbados,Nationality_Belarus,Nationality_Belgium,Nationality_Belize,Nationality_Benin,Nationality_Bermuda,Nationality_Bolivia,Nationality_Bosnia Herzegovina,Nationality_Botswana,Nationality_Brazil,Nationality_Bulgaria,Nationality_Burkina Faso,Nationality_Burundi,Nationality_Cameroon,Nationality_Canada,Nationality_Cape Verde,Nationality_Central African Rep.,Nationality_Chad,Nationality_Chile,Nationality_China PR,Nationality_Colombia,Nationality_Comoros,Nationality_Congo,Nationality_Costa Rica,Nationality_Croatia,Nationality_Cuba,Nationality_Curacao,Nationality_Cyprus,Nationality_Czech Republic,Nationality_DR Congo,Nationality_Denmark,Nationality_Dominican Republic,Nationality_Ecuador,Nationality_Egypt,Nationality_El Salvador,Nationality_England,Nationality_Equatorial Guinea,Nationality_Eritrea,Nationality_Estonia,Nationality_Ethiopia,Nationality_FYR Macedonia,Nationality_Faroe Islands,Nationality_Fiji,Nationality_Finland,Nationality_France,Nationality_Gabon,Nationality_Gambia,Nationality_Georgia,Nationality_Germany,Nationality_Ghana,Nationality_Greece,Nationality_Grenada,Nationality_Guam,Nationality_Guatemala,Nationality_Guinea,Nationality_Guinea Bissau,Nationality_Guyana,Nationality_Haiti,Nationality_Honduras,Nationality_Hong Kong,Nationality_Hungary,Nationality_Iceland,Nationality_India,Nationality_Indonesia,Nationality_Iran,Nationality_Iraq,Nationality_Israel,Nationality_Italy,Nationality_Ivory Coast,Nationality_Jamaica,Nationality_Japan,Nationality_Jordan,Nationality_Kazakhstan,Nationality_Kenya,Nationality_Korea DPR,Nationality_Korea Republic,Nationality_Kosovo,Nationality_Kuwait,Nationality_Latvia,Nationality_Lebanon,Nationality_Liberia,Nationality_Libya,Nationality_Liechtenstein,Nationality_Lithuania,Nationality_Luxembourg,Nationality_Madagascar,Nationality_Mali,Nationality_Malta,Nationality_Mauritania,Nationality_Mauritius,Nationality_Mexico,Nationality_Moldova,Nationality_Montenegro,Nationality_Montserrat,Nationality_Morocco,Nationality_Mozambique,Nationality_Namibia,Nationality_Netherlands,Nationality_New Caledonia,Nationality_New Zealand,Nationality_Nicaragua,Nationality_Niger,Nationality_Nigeria,Nationality_Northern Ireland,Nationality_Norway,Nationality_Oman,Nationality_Palestine,Nationality_Panama,Nationality_Paraguay,Nationality_Peru,Nationality_Philippines,Nationality_Poland,Nationality_Portugal,Nationality_Puerto Rico,Nationality_Qatar,Nationality_Republic of Ireland,Nationality_Romania,Nationality_Russia,Nationality_Rwanda,Nationality_Saudi Arabia,Nationality_Scotland,Nationality_Senegal,Nationality_Serbia,Nationality_Sierra Leone,Nationality_Slovakia,Nationality_Slovenia,Nationality_South Africa,Nationality_South Sudan,Nationality_Spain,Nationality_St Kitts Nevis,Nationality_St Lucia,Nationality_Sudan,Nationality_Suriname,Nationality_Sweden,Nationality_Switzerland,Nationality_Syria,Nationality_São Tomé & Príncipe,Nationality_Tanzania,Nationality_Thailand,Nationality_Togo,Nationality_Trinidad & Tobago,Nationality_Tunisia,Nationality_Turkey,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationality_Uzbekistan,Nat

Queremos hacer lo mismo con otras columnas que contienen valores no numéricos, pero que al relacionarlas con otras características nos dan información importante. Estas son: **Work Rate**, **Body Type** y **Position**.

In [26]:
df.loc[df["Work Rate"] != df["Work Rate"]]

,Name,Age,Overall,Potential,Value,Wage,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Nationality_Afghanistan,Nationality_Albania,Nationality_Algeria,Nationality_Andorra,Nationality_Angola,Nationality_Antigua & Barbuda,Nationality_Argentina,Nationality_Armenia,Nationality_Australia,Nationality_Austria,Nationality_Azerbaijan,Nationality_Barbados,Nationality_Belarus,Nationality_Belgium,Nationality_Belize,Nationality_Benin,Nationality_Bermuda,Nationality_Bolivia,Nationality_Bosnia Herzegovina,Nationality_Botswana,Nationality_Brazil,Nationality_Bulgaria,Nationality_Burkina Faso,Nationality_Burundi,Nationality_Cameroon,Nationality_Canada,Nationality_Cape Verde,Nationality_Central African Rep.,Nationality_Chad,Nationality_Chile,Nationality_China PR,Nationality_Colombia,Nationality_Comoros,Nationality_Congo,Nationality_Costa Rica,Nationality_Croatia,Nationality_Cuba,Nationality_Curacao,Nationality_Cyprus,Nationality_Czech Republic,Nationality_DR Congo,Nationality_Denmark,Nationality_Dominican Republic,Nationality_Ecuador,Nationality_Egypt,Nationality_El Salvador,Nationality_England,Nationality_Equatorial Guinea,Nationality_Eritrea,Nationality_Estonia,Nationality_Ethiopia,Nationality_FYR Macedonia,Nationality_Faroe Islands,Nationality_Fiji,Nationality_Finland,Nationality_France,Nationality_Gabon,Nationality_Gambia,Nationality_Georgia,Nationality_Germany,Nationality_Ghana,Nationality_Greece,Nationality_Grenada,Nationality_Guam,Nationality_Guatemala,Nationality_Guinea,Nationality_Guinea Bissau,Nationality_Guyana,Nationality_Haiti,Nationality_Honduras,Nationality_Hong Kong,Nationality_Hungary,Nationality_Iceland,Nationality_India,Nationality_Indonesia,Nationality_Iran,Nationality_Iraq,Nationality_Israel,Nationality_Italy,Nationality_Ivory Coast,Nationality_Jamaica,Nationality_Japan,Nationality_Jordan,Nationality_Kazakhstan,Nationality_Kenya,Nationality_Korea DPR,Nationality_Korea Republic,Nationality_Kosovo,Nationality_Kuwait,Nationality_Latvia,Nationality_Lebanon,Nationality_Liberia,Nationality_Libya,Nationality_Liechtenstein,Nationality_Lithuania,Nationality_Luxembourg,Nationality_Madagascar,Nationality_Mali,Nationality_Malta,Nationality_Mauritania,Nationality_Mauritius,Nationality_Mexico,Nationality_Moldova,Nationality_Montenegro,Nationality_Montserrat,Nationality_Morocco,Nationality_Mozambique,Nationality_Namibia,Nationality_Netherlands,Nationality_New Caledonia,Nationality_New Zealand,Nationality_Nicaragua,Nationality_Niger,Nationality_Nigeria,Nationality_Northern Ireland,Nationality_Norway,Nationality_Oman,Nationality_Palestine,Nationality_Panama,Nationality_Paraguay,Nationality_Peru,Nationality_Philippines,Nationality_Poland,Nationality_Portugal,Nationality_Puerto Rico,Nationality_Qatar,Nationality_Republic of Ireland,Nationality_Romania,Nationality_Russia,Nationality_Rwanda,Nationality_Saudi Arabia,Nationality_Scotland,Nationality_Senegal,Nationality_Serbia,Nationality_Sierra Leone,Nationality_Slovakia,Nationality_Slovenia,Nationality_South Africa,Nationality_South Sudan,Nationality_Spain,Nationality_St Kitts Nevis,Nationality_St Lucia,Nationality_Sudan,Nationality_Suriname,Nationality_Sweden,Nationality_Switzerland,Nationality_Syria,Nationality_São Tomé & Príncipe,Nationality_Tanzania,Nationality_Thailand,Nationality_Togo,Nationality_Trinidad & Tobago,Nationality_Tunisia,Nationality_Turkey,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationality_Uzbekistan,Nat

In [27]:
df = df.dropna(subset = ['Work Rate'])

In [28]:
wrate = df.pop("Work Rate")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(wrate, prefix='w_rate').reset_index(drop=True)], axis=1, sort=False)
df.head()

,Name,Age,Overall,Potential,Value,Wage,International Reputation,Weak Foot,Skill Moves,Body Type,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Nationality_Afghanistan,Nationality_Albania,Nationality_Algeria,Nationality_Andorra,Nationality_Angola,Nationality_Antigua & Barbuda,Nationality_Argentina,Nationality_Armenia,Nationality_Australia,Nationality_Austria,Nationality_Azerbaijan,Nationality_Barbados,Nationality_Belarus,Nationality_Belgium,Nationality_Belize,Nationality_Benin,Nationality_Bermuda,Nationality_Bolivia,Nationality_Bosnia Herzegovina,Nationality_Botswana,Nationality_Brazil,Nationality_Bulgaria,Nationality_Burkina Faso,Nationality_Burundi,Nationality_Cameroon,Nationality_Canada,Nationality_Cape Verde,Nationality_Central African Rep.,Nationality_Chad,Nationality_Chile,Nationality_China PR,Nationality_Colombia,Nationality_Comoros,Nationality_Congo,Nationality_Costa Rica,Nationality_Croatia,Nationality_Cuba,Nationality_Curacao,Nationality_Cyprus,Nationality_Czech Republic,Nationality_DR Congo,Nationality_Denmark,Nationality_Dominican Republic,Nationality_Ecuador,Nationality_Egypt,Nationality_El Salvador,Nationality_England,Nationality_Equatorial Guinea,Nationality_Eritrea,Nationality_Estonia,Nationality_Ethiopia,Nationality_FYR Macedonia,Nationality_Faroe Islands,Nationality_Fiji,Nationality_Finland,Nationality_France,Nationality_Gabon,Nationality_Gambia,Nationality_Georgia,Nationality_Germany,Nationality_Ghana,Nationality_Greece,Nationality_Grenada,Nationality_Guam,Nationality_Guatemala,Nationality_Guinea,Nationality_Guinea Bissau,Nationality_Guyana,Nationality_Haiti,Nationality_Honduras,Nationality_Hong Kong,Nationality_Hungary,Nationality_Iceland,Nationality_India,Nationality_Indonesia,Nationality_Iran,Nationality_Iraq,Nationality_Israel,Nationality_Italy,Nationality_Ivory Coast,Nationality_Jamaica,Nationality_Japan,Nationality_Jordan,Nationality_Kazakhstan,Nationality_Kenya,Nationality_Korea DPR,Nationality_Korea Republic,Nationality_Kosovo,Nationality_Kuwait,Nationality_Latvia,Nationality_Lebanon,Nationality_Liberia,Nationality_Libya,Nationality_Liechtenstein,Nationality_Lithuania,Nationality_Luxembourg,Nationality_Madagascar,Nationality_Mali,Nationality_Malta,Nationality_Mauritania,Nationality_Mauritius,Nationality_Mexico,Nationality_Moldova,Nationality_Montenegro,Nationality_Montserrat,Nationality_Morocco,Nationality_Mozambique,Nationality_Namibia,Nationality_Netherlands,Nationality_New Caledonia,Nationality_New Zealand,Nationality_Nicaragua,Nationality_Niger,Nationality_Nigeria,Nationality_Northern Ireland,Nationality_Norway,Nationality_Oman,Nationality_Palestine,Nationality_Panama,Nationality_Paraguay,Nationality_Peru,Nationality_Philippines,Nationality_Poland,Nationality_Portugal,Nationality_Puerto Rico,Nationality_Qatar,Nationality_Republic of Ireland,Nationality_Romania,Nationality_Russia,Nationality_Rwanda,Nationality_Saudi Arabia,Nationality_Scotland,Nationality_Senegal,Nationality_Serbia,Nationality_Sierra Leone,Nationality_Slovakia,Nationality_Slovenia,Nationality_South Africa,Nationality_South Sudan,Nationality_Spain,Nationality_St Kitts Nevis,Nationality_St Lucia,Nationality_Sudan,Nationality_Suriname,Nationality_Sweden,Nationality_Switzerland,Nationality_Syria,Nationality_São Tomé & Príncipe,Nationality_Tanzania,Nationality_Thailand,Nationality_Togo,Nationality_Trinidad & Tobago,Nationality_Tunisia,Nationality_Turkey,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationality_Uzbekistan,Nationality_V

In [29]:
btype = df.pop("Body Type")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(btype, prefix='b_type').reset_index(drop=True)], axis=1, sort=False)
df.head()

,Name,Age,Overall,Potential,Value,Wage,International Reputation,Weak Foot,Skill Moves,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Nationality_Afghanistan,Nationality_Albania,Nationality_Algeria,Nationality_Andorra,Nationality_Angola,Nationality_Antigua & Barbuda,Nationality_Argentina,Nationality_Armenia,Nationality_Australia,Nationality_Austria,Nationality_Azerbaijan,Nationality_Barbados,Nationality_Belarus,Nationality_Belgium,Nationality_Belize,Nationality_Benin,Nationality_Bermuda,Nationality_Bolivia,Nationality_Bosnia Herzegovina,Nationality_Botswana,Nationality_Brazil,Nationality_Bulgaria,Nationality_Burkina Faso,Nationality_Burundi,Nationality_Cameroon,Nationality_Canada,Nationality_Cape Verde,Nationality_Central African Rep.,Nationality_Chad,Nationality_Chile,Nationality_China PR,Nationality_Colombia,Nationality_Comoros,Nationality_Congo,Nationality_Costa Rica,Nationality_Croatia,Nationality_Cuba,Nationality_Curacao,Nationality_Cyprus,Nationality_Czech Republic,Nationality_DR Congo,Nationality_Denmark,Nationality_Dominican Republic,Nationality_Ecuador,Nationality_Egypt,Nationality_El Salvador,Nationality_England,Nationality_Equatorial Guinea,Nationality_Eritrea,Nationality_Estonia,Nationality_Ethiopia,Nationality_FYR Macedonia,Nationality_Faroe Islands,Nationality_Fiji,Nationality_Finland,Nationality_France,Nationality_Gabon,Nationality_Gambia,Nationality_Georgia,Nationality_Germany,Nationality_Ghana,Nationality_Greece,Nationality_Grenada,Nationality_Guam,Nationality_Guatemala,Nationality_Guinea,Nationality_Guinea Bissau,Nationality_Guyana,Nationality_Haiti,Nationality_Honduras,Nationality_Hong Kong,Nationality_Hungary,Nationality_Iceland,Nationality_India,Nationality_Indonesia,Nationality_Iran,Nationality_Iraq,Nationality_Israel,Nationality_Italy,Nationality_Ivory Coast,Nationality_Jamaica,Nationality_Japan,Nationality_Jordan,Nationality_Kazakhstan,Nationality_Kenya,Nationality_Korea DPR,Nationality_Korea Republic,Nationality_Kosovo,Nationality_Kuwait,Nationality_Latvia,Nationality_Lebanon,Nationality_Liberia,Nationality_Libya,Nationality_Liechtenstein,Nationality_Lithuania,Nationality_Luxembourg,Nationality_Madagascar,Nationality_Mali,Nationality_Malta,Nationality_Mauritania,Nationality_Mauritius,Nationality_Mexico,Nationality_Moldova,Nationality_Montenegro,Nationality_Montserrat,Nationality_Morocco,Nationality_Mozambique,Nationality_Namibia,Nationality_Netherlands,Nationality_New Caledonia,Nationality_New Zealand,Nationality_Nicaragua,Nationality_Niger,Nationality_Nigeria,Nationality_Northern Ireland,Nationality_Norway,Nationality_Oman,Nationality_Palestine,Nationality_Panama,Nationality_Paraguay,Nationality_Peru,Nationality_Philippines,Nationality_Poland,Nationality_Portugal,Nationality_Puerto Rico,Nationality_Qatar,Nationality_Republic of Ireland,Nationality_Romania,Nationality_Russia,Nationality_Rwanda,Nationality_Saudi Arabia,Nationality_Scotland,Nationality_Senegal,Nationality_Serbia,Nationality_Sierra Leone,Nationality_Slovakia,Nationality_Slovenia,Nationality_South Africa,Nationality_South Sudan,Nationality_Spain,Nationality_St Kitts Nevis,Nationality_St Lucia,Nationality_Sudan,Nationality_Suriname,Nationality_Sweden,Nationality_Switzerland,Nationality_Syria,Nationality_São Tomé & Príncipe,Nationality_Tanzania,Nationality_Thailand,Nationality_Togo,Nationality_Trinidad & Tobago,Nationality_Tunisia,Nationality_Turkey,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationality_Uzbekistan,Nationality_Venezuela,N

In [30]:
position = df.pop("Position")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(position, prefix='position').reset_index(drop=True)], axis=1, sort=False)
df.head()

,Name,Age,Overall,Potential,Value,Wage,International Reputation,Weak Foot,Skill Moves,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Nationality_Afghanistan,Nationality_Albania,Nationality_Algeria,Nationality_Andorra,Nationality_Angola,Nationality_Antigua & Barbuda,Nationality_Argentina,Nationality_Armenia,Nationality_Australia,Nationality_Austria,Nationality_Azerbaijan,Nationality_Barbados,Nationality_Belarus,Nationality_Belgium,Nationality_Belize,Nationality_Benin,Nationality_Bermuda,Nationality_Bolivia,Nationality_Bosnia Herzegovina,Nationality_Botswana,Nationality_Brazil,Nationality_Bulgaria,Nationality_Burkina Faso,Nationality_Burundi,Nationality_Cameroon,Nationality_Canada,Nationality_Cape Verde,Nationality_Central African Rep.,Nationality_Chad,Nationality_Chile,Nationality_China PR,Nationality_Colombia,Nationality_Comoros,Nationality_Congo,Nationality_Costa Rica,Nationality_Croatia,Nationality_Cuba,Nationality_Curacao,Nationality_Cyprus,Nationality_Czech Republic,Nationality_DR Congo,Nationality_Denmark,Nationality_Dominican Republic,Nationality_Ecuador,Nationality_Egypt,Nationality_El Salvador,Nationality_England,Nationality_Equatorial Guinea,Nationality_Eritrea,Nationality_Estonia,Nationality_Ethiopia,Nationality_FYR Macedonia,Nationality_Faroe Islands,Nationality_Fiji,Nationality_Finland,Nationality_France,Nationality_Gabon,Nationality_Gambia,Nationality_Georgia,Nationality_Germany,Nationality_Ghana,Nationality_Greece,Nationality_Grenada,Nationality_Guam,Nationality_Guatemala,Nationality_Guinea,Nationality_Guinea Bissau,Nationality_Guyana,Nationality_Haiti,Nationality_Honduras,Nationality_Hong Kong,Nationality_Hungary,Nationality_Iceland,Nationality_India,Nationality_Indonesia,Nationality_Iran,Nationality_Iraq,Nationality_Israel,Nationality_Italy,Nationality_Ivory Coast,Nationality_Jamaica,Nationality_Japan,Nationality_Jordan,Nationality_Kazakhstan,Nationality_Kenya,Nationality_Korea DPR,Nationality_Korea Republic,Nationality_Kosovo,Nationality_Kuwait,Nationality_Latvia,Nationality_Lebanon,Nationality_Liberia,Nationality_Libya,Nationality_Liechtenstein,Nationality_Lithuania,Nationality_Luxembourg,Nationality_Madagascar,Nationality_Mali,Nationality_Malta,Nationality_Mauritania,Nationality_Mauritius,Nationality_Mexico,Nationality_Moldova,Nationality_Montenegro,Nationality_Montserrat,Nationality_Morocco,Nationality_Mozambique,Nationality_Namibia,Nationality_Netherlands,Nationality_New Caledonia,Nationality_New Zealand,Nationality_Nicaragua,Nationality_Niger,Nationality_Nigeria,Nationality_Northern Ireland,Nationality_Norway,Nationality_Oman,Nationality_Palestine,Nationality_Panama,Nationality_Paraguay,Nationality_Peru,Nationality_Philippines,Nationality_Poland,Nationality_Portugal,Nationality_Puerto Rico,Nationality_Qatar,Nationality_Republic of Ireland,Nationality_Romania,Nationality_Russia,Nationality_Rwanda,Nationality_Saudi Arabia,Nationality_Scotland,Nationality_Senegal,Nationality_Serbia,Nationality_Sierra Leone,Nationality_Slovakia,Nationality_Slovenia,Nationality_South Africa,Nationality_South Sudan,Nationality_Spain,Nationality_St Kitts Nevis,Nationality_St Lucia,Nationality_Sudan,Nationality_Suriname,Nationality_Sweden,Nationality_Switzerland,Nationality_Syria,Nationality_São Tomé & Príncipe,Nationality_Tanzania,Nationality_Thailand,Nationality_Togo,Nationality_Trinidad & Tobago,Nationality_Tunisia,Nationality_Turkey,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationality_Uzbekistan,Nationality_Venezuela,Nationalit

In [31]:
df.columns[df.isna().any()].tolist()

['LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM',
 'CAM',
 'RAM',
 'LM',
 'LCM',
 'CM',
 'RCM',
 'RM',
 'LWB',
 'LDM',
 'CDM',
 'RDM',
 'RWB',
 'LB',
 'LCB',
 'CB',
 'RCB',
 'RB']

In [32]:
df.loc[:, 'LS':'RB']

,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB
0,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2
1,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
2,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17913,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2
17914,45+2,45+2,45+2,39+2,42+2,42+2,42+2,39+2,40+2,40+2,40+2,38+2,35+2,35+2,35+2,38+2,30+2,31+2,31+2,31+2,30+2,29+2,32+2,32+2,32+2,29+2
17915,45+2,45+2,45+2,45+2,46+2,46+2,46+2,45+2,44+2,44+2,44+2,44+2,38+2,38+2,38+2,44+2,34+2,30+2,30+2,30+2,34+2,33+2,28+2,28+2,28+2,33+2
17916,47+2,47+2,47+2,47+2,46+2,46+2,46+2,47+2,45+2,45+2,45+2,46+2,39+2,39+2,39+2,46+2,36+2,32+2,32+2,32+2,36+2,35+2,31+2,31+2,31+2,35+2


In [33]:
list = df.loc[:, 'LS':'RB'].columns.tolist()

In [34]:
def add(x):
    """
    Add 
    
    """
    if x=="nan":
        return
    else:
        x = x.split('+')
        ret_val = int(x[0])+int(x[1])
        return ret_val

In [35]:
for i in range(len(list)):
    df[list[i]] = df[list[i]].astype(str)
    df[list[i]] = df[list[i]].apply(add)

In [36]:
df.loc[:, 'LS':'RB']

,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB
0,90.0,90.0,90.0,94.0,95.0,95.0,95.0,94.0,95.0,95.0,95.0,93.0,86.0,86.0,86.0,93.0,66.0,63.0,63.0,63.0,66.0,61.0,49.0,49.0,49.0,61.0
1,94.0,94.0,94.0,92.0,93.0,93.0,93.0,92.0,91.0,91.0,91.0,91.0,84.0,84.0,84.0,91.0,68.0,64.0,64.0,64.0,68.0,64.0,56.0,56.0,56.0,64.0
2,87.0,87.0,87.0,92.0,92.0,92.0,92.0,92.0,92.0,92.0,92.0,91.0,84.0,84.0,84.0,91.0,68.0,63.0,63.0,63.0,68.0,63.0,50.0,50.0,50.0,63.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,85.0,85.0,85.0,90.0,90.0,90.0,90.0,90.0,91.0,91.0,91.0,91.0,90.0,90.0,90.0,91.0,80.0,80.0,80.0,80.0,80.0,76.0,69.0,69.0,69.0,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17913,44.0,44.0,44.0,46.0,46.0,46.0,46.0,46.0,47.0,47.0,47.0,46.0,47.0,47.0,47.0,46.0,46.0,47.0,47.0,47.0,46.0,47.0,47.0,47.0,47.0,47.0
17914,47.0,47.0,47.0,41.0,44.0,44.0,44.0,41.0,42.0,42.0,42.0,40.0,37.0,37.0,37.0,40.0,32.0,33.0,33.0,33.0,32.0,31.0,34.0,34.0,34.0,31.0
17915,47.0,47.0,47.0,47.0,48.0,48.0,48.0,47.0,46.0,46.0,46.0,46.0,40.0,40.0,40.0,46.0,36.0,32.0,32.0,32.0,36.0,35.0,30.0,30.0,30.0,35.0
17916,49.0,49.0,49.0,49.0,48.0,48.0,48.0,49.0,47.0,47.0,47.0,48.0,41.0,41.0,41.0,48.0,38.0,34.0,34.0,34.0,38.0,37.0,33.0,33.0,33.0,37.0


Para cada una de estas columnas hallaremos la media, que es un valor razonable para sustituir los NaN que hay en cada una.

In [37]:
for i in range(len(list)):
    mean = df[list[i]].mean()
    df[list[i]] = df[list[i]].fillna(mean)

In [38]:
df

,Name,Age,Overall,Potential,Value,Wage,International Reputation,Weak Foot,Skill Moves,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Nationality_Afghanistan,Nationality_Albania,Nationality_Algeria,Nationality_Andorra,Nationality_Angola,Nationality_Antigua & Barbuda,Nationality_Argentina,Nationality_Armenia,Nationality_Australia,Nationality_Austria,Nationality_Azerbaijan,Nationality_Barbados,Nationality_Belarus,Nationality_Belgium,Nationality_Belize,Nationality_Benin,Nationality_Bermuda,Nationality_Bolivia,Nationality_Bosnia Herzegovina,Nationality_Botswana,Nationality_Brazil,Nationality_Bulgaria,Nationality_Burkina Faso,Nationality_Burundi,Nationality_Cameroon,Nationality_Canada,Nationality_Cape Verde,Nationality_Central African Rep.,Nationality_Chad,Nationality_Chile,Nationality_China PR,Nationality_Colombia,Nationality_Comoros,Nationality_Congo,Nationality_Costa Rica,Nationality_Croatia,Nationality_Cuba,Nationality_Curacao,Nationality_Cyprus,Nationality_Czech Republic,Nationality_DR Congo,Nationality_Denmark,Nationality_Dominican Republic,Nationality_Ecuador,Nationality_Egypt,Nationality_El Salvador,Nationality_England,Nationality_Equatorial Guinea,Nationality_Eritrea,Nationality_Estonia,Nationality_Ethiopia,Nationality_FYR Macedonia,Nationality_Faroe Islands,Nationality_Fiji,Nationality_Finland,Nationality_France,Nationality_Gabon,Nationality_Gambia,Nationality_Georgia,Nationality_Germany,Nationality_Ghana,Nationality_Greece,Nationality_Grenada,Nationality_Guam,Nationality_Guatemala,Nationality_Guinea,Nationality_Guinea Bissau,Nationality_Guyana,Nationality_Haiti,Nationality_Honduras,Nationality_Hong Kong,Nationality_Hungary,Nationality_Iceland,Nationality_India,Nationality_Indonesia,Nationality_Iran,Nationality_Iraq,Nationality_Israel,Nationality_Italy,Nationality_Ivory Coast,Nationality_Jamaica,Nationality_Japan,Nationality_Jordan,Nationality_Kazakhstan,Nationality_Kenya,Nationality_Korea DPR,Nationality_Korea Republic,Nationality_Kosovo,Nationality_Kuwait,Nationality_Latvia,Nationality_Lebanon,Nationality_Liberia,Nationality_Libya,Nationality_Liechtenstein,Nationality_Lithuania,Nationality_Luxembourg,Nationality_Madagascar,Nationality_Mali,Nationality_Malta,Nationality_Mauritania,Nationality_Mauritius,Nationality_Mexico,Nationality_Moldova,Nationality_Montenegro,Nationality_Montserrat,Nationality_Morocco,Nationality_Mozambique,Nationality_Namibia,Nationality_Netherlands,Nationality_New Caledonia,Nationality_New Zealand,Nationality_Nicaragua,Nationality_Niger,Nationality_Nigeria,Nationality_Northern Ireland,Nationality_Norway,Nationality_Oman,Nationality_Palestine,Nationality_Panama,Nationality_Paraguay,Nationality_Peru,Nationality_Philippines,Nationality_Poland,Nationality_Portugal,Nationality_Puerto Rico,Nationality_Qatar,Nationality_Republic of Ireland,Nationality_Romania,Nationality_Russia,Nationality_Rwanda,Nationality_Saudi Arabia,Nationality_Scotland,Nationality_Senegal,Nationality_Serbia,Nationality_Sierra Leone,Nationality_Slovakia,Nationality_Slovenia,Nationality_South Africa,Nationality_South Sudan,Nationality_Spain,Nationality_St Kitts Nevis,Nationality_St Lucia,Nationality_Sudan,Nationality_Suriname,Nationality_Sweden,Nationality_Switzerland,Nationality_Syria,Nationality_São Tomé & Príncipe,Nationality_Tanzania,Nationality_Thailand,Nationality_Togo,Nationality_Trinidad & Tobago,Nationality_Tunisia,Nationality_Turkey,Nationality_Uganda,Nationality_Ukraine,Nationality_United Arab Emirates,Nationality_United States,Nationality_Uruguay,Nationality_Uzbekistan,Nationality_Venezuela,Nationalit

Finalmente, ya nos hemos deshecho de los valores NaN presentes en nuestro set de datos.
Sin embargo, hasta el momento habíamos mantenido la columna **Name** por tratarse de un valor informativo y de referencia a lo largo del análisis y tratamiento de los datos, pero por tratarse de un dato no numérico, será eliminado del set.

In [39]:
df.pop('Name')

0                  L. Messi
1         Cristiano Ronaldo
2                 Neymar Jr
3                    De Gea
4              K. De Bruyne
                ...        
17913          J. Lundstram
17914    N. Christoffersson
17915             B. Worman
17916        D. Walker-Rice
17917             G. Nugent
Name: Name, Length: 17918, dtype: object

In [41]:
val = df["Value"]

## Predicción

En este punto, ya los datos están limpios y en el formato que nuestro algoritmo acepta. El siguiente paso es entrenar y predecir, pero antes debemos dividir el set de datos en: entrenamiento y test. La proporción será 80% y 20%, respectivamente.

In [42]:
X_train, X_test, y_train, y_test = train_test_split(df,df["Value"], test_size=0.20, random_state=50)

Ahora que ya el set está dividido, procedemos a normalizar los datos.

In [43]:
norm = preprocessing.MinMaxScaler()
X_train_norm = norm.fit_transform(X_train)

In [44]:
X_test_norm = norm.transform(X_test)

Finalmente, hacemos la **regresión lineal** para obtener la métrica $R^2$

In [45]:
reg = linear_model.LinearRegression().fit(X_train, y_train)

In [46]:
preds = reg.predict(X_test)

In [47]:
mean_squared_error(y_test, preds)

3.4760104287989156e-17

In [48]:
r2_score(preds, y_test)

1.0

Sabemos que para las regresiones que si la métrica obtenida es cercana o igual a 1, se trata de un ajuste bastante bueno y, por lo tanto, las predicciones que se hagan con ese modelo serán bastante correctas.

Así, al haber obtenido una métrica igual a 1, concluimos que las predicciones que se hacen este algoritmo sobre el valor monetario de un jugador de fútbol son fiables.